In [1]:
import re 


Slabel = re.compile(r'[A-Z][0-9]+')
Costos = re.compile(r'[0-9]+')

u=0
m=0
Q=0 

#cambiar----------------------------------------------------------
rutas=247
nodos=8
fic = open("BRT-C1-S8-GENERADO.txt", "r")

#cambiar----------------------------------------------------------
lines = []
for line in fic:
    lines.append(line)

#leer Slabel
i=0
while(lines[i]!="Slabel:[\n"):
    i=i+1

i=i+1 
nodos2={}
j=i
while(lines[i]!="]\n") :
    result = Slabel.search(lines[i])
  ##print(lines[i]) 
    i=i+1 
    nodos2[i-j]= result.group()

i=i+1  




#leer Dlabel
while(lines[i]!="Dlabel:[\n"):
    i=i+1
i=i+1  
while(lines[i]!="]\n") :
    i=i+1 
i=i+1 

#leer costos
c2={}
while(lines[i]!="cost:[\n"):
    i=i+1
i=i+1  
while(lines[i]!="]\n") :
    result = Costos.findall(lines[i]) 
    i=i+1
    c2[(int(result[0]),int(result[1]))]=int(result[2])
i=i+1 



#leer u
while(lines[i][0]!="u"):
    i=i+1
result = Costos.findall(lines[i])   
u=int(result[0])
 
i=i+1
#leer m

while(lines[i][0]!="m"):
    i=i+1
result = Costos.findall(lines[i])   
m=int(result[0])
 
i=i+1  

#leer Q
while(lines[i][0]!="Q"):
    i=i+1
result = Costos.findall(lines[i])   
Q=int(result[0])

i=i+1   

rutas2={(i,j):0 for i in range(1,rutas+1) for j in range(1,nodos+1)}
#leer stat.
stat={}
while(lines[i]!="stat:[\n"):
    i=i+1
i=i+1  
while(lines[i]!="]\n") :
    result = Costos.findall(lines[i]) 
    i=i+1 
    stat[int(result[0])]= int(result[1]) 
i=i+1 

B2={(o,d,i):0 for o in nodos2 for d in nodos2 for i in stat}


#leer b
while(lines[i]!="b:[\n"):
    i=i+1
i=i+1  
while(lines[i]!="]\n") :
    result = Costos.findall(lines[i]) 
    B2[(int(result[0]),int(result[1]),int(result[0]))]=int(result[2])
    B2[(int(result[0]),int(result[1]),int(result[1]))]=(-1)*int(result[2])  
    i=i+1 
i=i+1 

Ar2={}
rutaAsociada={}

#leer< Ar
while(lines[i]!="Ar:[\n"):
    i=i+1
i=i+1  
j=i
#cambia segun experimento-------------------------------------------------------------------<


while(lines[i]!="]\n") :
    result = Costos.findall(lines[i]) 
    i=i+1 
    Ar2[(int(result[0]),int(result[1]))]=int(result[3])
    rutaAsociada[i-j]=int(result[2])
    rutas2[(int(result[2]),stat[int(result[0])])]=1
    rutas2[(int(result[2]),stat[int(result[1])])]=1


i=i+1 



As2={}

#leer As
while(lines[i]!="As:[\n"):
    i=i+1
i=i+1  
while(lines[i]!="]\n") :
    result = Costos.findall(lines[i]) 
    i=i+1 
    As2[(int(result[0]),int(result[1]))]=int(result[3]) 
i=i+1 

#matriz de costos nueva

C2={}

for i in Ar2:
    C2[i]=c2[(stat[i[0]],stat[i[1]])]  
for i in As2:   
    C2[i]=10


In [2]:

from docplex.mp.model import Model

mdl= Model("modelo")
arcos=[(i[0],i[1],o,d) for i in C2 for o in nodos2 for d in nodos2]
x=mdl.continuous_var_dict(arcos,name="x")

arcosF=[(i) for i in range(1,rutas+1)]
f=mdl.continuous_var_dict(arcosF,name="f")
y=mdl.binary_var_dict(arcosF,name="y")


u= 20
m= 60
Q= 50
alpha=100


mdl.minimize(mdl.sum(C2[i]*x[(i[0],i[1],o,d)] for i in C2 for o in nodos2 for d in nodos2)+alpha*mdl.sum(f[k]+y[k] for k in range(1,rutas+1)))

aux={}
aux2={}
for i in stat:
    aux.clear()  
    aux2.clear()
    for j in C2:
        if(i==j[0]):
            aux[j]=C2[j]
    for j in C2:
        if(i==j[1]):
            aux2[j]=C2[j]
                        
    for o in  nodos2:
        for d in nodos2:       
            mdl.add_constraint((mdl.sum(x[(k[0],k[1],o,d) ] for k in aux )-mdl.sum(x[(k[0],k[1],o,d) ] for k in aux2 ) )==B2[(o,d,i)])


aux3={}
for k in range(1,rutas+1):
    j=1
    aux3.clear()
    for i in Ar2:
        if(k==rutaAsociada[j]):
            aux3[i]=1            
        j=j+1
    for l in aux3:    
        mdl.add_constraint(mdl.sum(x[(l[0],l[1],o,d)] for o in nodos2 for d in nodos2 )<=u*f[(k)])  
        
for k in range(1,rutas+1):
    mdl.add_constraint(f[(k)]<=y[(k)]*10)

mdl.add_constraint(mdl.sum(y[(k)] for k in range(1,rutas+1))<=m) 
mdl.add_constraint(mdl.sum(f[k] for k in range(1,rutas+1))<=Q)    

docplex.mp.LinearConstraint[](f_1+f_2+f_3+f_4+f_5+f_6+f_7+f_8+f_9+f_10+f_11+f_12+f_13+f_14+f_15+f_16+f_17+f_18+f_19+f_20+f_21+f_22+f_23+f_24+f_25+f_26+f_27+f_28+f_29+f_30+f_31+f_32+f_33+f_34+f_35+f_36+f_37+f_38+f_39+f_40+f_41+f_42+f_43+f_44+f_45+f_46+f_47+f_48+f_49+f_50+f_51+f_52+f_53+f_54+f_55+f_56+f_57+f_58+f_59+f_60+f_61+f_62+f_63+f_64+f_65+f_66+f_67+f_68+f_69+f_70+f_71+f_72+f_73+f_74+f_75+f_76+f_77+f_78+f_79+f_80+f_81+f_82+f_83+f_84+f_85+f_86+f_87+f_88+f_89+f_90+f_91+f_92+f_93+f_94+f_95+f_96+f_97+f_98+f_99+f_100+f_101+f_102+f_103+f_104+f_105+f_106+f_107+f_108+f_109+f_110+f_111+f_112+f_113+f_114+f_115+f_116+f_117+f_118+f_119+f_120+f_121+f_122+f_123+f_124+f_125+f_126+f_127+f_128+f_129+f_130+f_131+f_132+f_133+f_134+f_135+f_136+f_137+f_138+f_139+f_140+f_141+f_142+f_143+f_144+f_145+f_146+f_147+f_148+f_149+f_150+f_151+f_152+f_153+f_154+f_155+f_156+f_157+f_158+f_159+f_160+f_161+f_162+f_163+f_164+f_165+f_166+f_167+f_168+f_169+f_170+f_171+f_172+f_173+f_174+f_175+f_176+f_177+f_178+f_179+f_18

In [ ]:
import time

inicio = time.time()
solucion=mdl.solve(log_output=True)  
fin = time.time()
print(fin-inicio)

archivo = open("completa-Experimento", "w") 
archivo.write(str(solucion)) 
archivo.close()
#m=10             

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Presolve has eliminated 0 rows and 0 columns...
Presolve has improved bounds 98679 times...
Tried aggregator 1 time.
Presolve has eliminated 0 rows and 0 columns...
Presolve has improved bounds 98679 times...
Reduced MIP has 67323 rows, 8421998 columns, and 16943966 nonzeros.
Reduced MIP has 247 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 34.59 sec. (29726.94 ticks)
Tried aggregator 1 time.
Detecting symmetries...


In [ ]:

for k in range(1,rutas+1):
    if(y[ k ].solution_value>0):
        print("rutas.append((",end="")
        for j in range(1,7):
            print(rutas2[(k,j)],end="")
            if(j!=6):
                print(",",end="")
            
        print("))",f[ k ].solution_value)
          

In [101]:
for k in range(1,rutas+1):
    if(y[ k ].solution_value>0):
        print("rutas.append((",end="")
        for j in range(1,5):
            print(rutas2[(k,j)],end="")
            if(j!=4):
                print(",",end="")
            
        print("))",f[ k ].solution_value)
        

    



rutas.append((1,0,1,1)) 1.1111111111111112
rutas.append((1,1,0,1)) 0.5555555555555556
rutas.append((1,1,1,1)) 0.5555555555555556


{1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9: 4, 10: 4, 11: 5, 12: 5, 13: 6, 14: 6, 15: 7, 16: 7, 17: 7, 18: 7, 19: 8, 20: 8, 21: 9, 22: 9, 23: 9, 24: 9, 25: 10, 26: 10, 27: 10, 28: 10, 29: 11, 30: 11, 31: 11, 32: 11, 33: 11, 34: 11}
